<a href="https://colab.research.google.com/github/SeongDeokKo/Computational-Economics/blob/main/Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd 
from collections import Counter
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/')
import numpy as np
import pandas as pd 
from collections import Counter
import FunLib_stock as FL
import multiprocessing as mp

In [5]:
new_firm_data = pd.read_csv('/content/drive/MyDrive/x_y_wo_inter.csv')

In [7]:
new_firm_data[new_firm_data.columns[2:96]] = new_firm_data[new_firm_data.columns[2:96]].astype('float32')
new_firm_data[new_firm_data.columns[0:2]] = new_firm_data[new_firm_data.columns[0:2]].astype('int32')
new_firm_data[new_firm_data.columns[96:170]] = new_firm_data[new_firm_data.columns[96:170]].astype('int8')
new_firm_data[new_firm_data.columns[170]] = new_firm_data[new_firm_data.columns[170]].astype('float32')

In [8]:

def R2OOS(y_true, y_forecast):
    
    import numpy as np
   
    SSres = np.nansum(np.square(y_true-y_forecast))
    SStot = np.nansum(np.square(y_true))

    return 1-SSres/SStot



# =========================================================================
#   PCR, 94 + dummy variable(no intersection term), Use cross-validation to select the number of PCA components  
# =========================================================================

def Pca_regression(X,Y,numpc,num_t_v):
    # numpc (list) : # of principal component ex[3,4,5,6,7]
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np 
    from sklearn.decomposition import PCA
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    
    num_train = num_t_v[0]
    num_val = num_t_v[1]
    num_test = X.shape[0] - (num_train + num_val)
    
    # Split data into training and test
    X_train = X[:num_train,:]
    Y_train = Y[:num_train,:]
    
    X_val = X[num_train:(num_train+num_val),:]
    Y_val = Y[num_train:(num_train+num_val),:]
    
    X_test = X[(num_train+num_val):,:]
    
       
    # Scale Inputs for Training
    X_scaler = StandardScaler()
    X_train_scaled = X_scaler.fit_transform(X_train)
    
    X_val_scaled = X_scaler.transform(X_val)
    X_test_scaled = X_scaler.transform(X_test)
    
    
    # use cross-validation mean-squared-error to determine the number of component 
    mse = np.full((len(numpc),1),np.nan)

    for i in range(len(numpc)):
        pca = PCA(n_components = numpc[i])
        principalComponents = pca.fit_transform(X_train_scaled)
        
        X_val_weighted = pca.transform(X_val_scaled)
        
        line_fitter = LinearRegression()
        line_fitter.fit(principalComponents, Y_train)
        
        Ypred_val = np.full((num_val,1),np.nan, dtype = np.float32)
        for j in range(num_val):
            Ypred_val[j,0] = line_fitter.predict(X_val_weighted[j,:].reshape(1,-1))
                   
        mse[i,0] = mean_squared_error(Y_val.reshape(-1), Ypred_val.reshape(-1))
    
    
    argmin_numpc = numpc[np.argmin(mse)]
    
    pca = PCA(n_components = argmin_numpc)
    principalComponents = pca.fit_transform(X_train_scaled)
    
    X_test_weighted = pca.transform(X_test_scaled)
    
    line_fitter = LinearRegression()
    line_fitter.fit(principalComponents, Y_train)
        
    Ypred_test = np.full((num_test,1),np.nan, dtype = np.float32)
    for j in range(num_test):
        Ypred_test[j,0]=line_fitter.predict(X_test_weighted[j,:].reshape(1,-1))
        
          
    return Ypred_test, argmin_numpc




# =========================================================================
#   PLS, 94 + dummy variable(no intersection term), Use cross-validation to select the number of components  
# =========================================================================

def Pls_regression(X,Y,numpls,num_t_v):
    # numpls (list) : # of component ex[3,4,5,6,7]
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np 
    from sklearn.cross_decomposition import PLSRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    
    num_train = num_t_v[0]
    num_val = num_t_v[1]
    num_test = X.shape[0] - (num_train + num_val)
    
    # Split data into training and test
    X_train = X[:num_train,:]
    Y_train = Y[:num_train,:]
    
    X_val = X[num_train:(num_train+num_val),:]
    Y_val = Y[num_train:(num_train+num_val),:]
    
    X_test = X[(num_train+num_val):,:]
    
       
    # Scale Inputs for Training
    X_scaler = StandardScaler()
    X_train_scaled = X_scaler.fit_transform(X_train)
    
    X_val_scaled = X_scaler.transform(X_val)
    X_test_scaled = X_scaler.transform(X_test)
    
    
    # use cross-validation mean-squared-error to determine the number of component 
    mse = np.full((len(numpls),1),np.nan)

    for i in range(len(numpls)):
        pls = PLSRegression(n_components = numpls[i])
        pls.fit(X_train_scaled, Y_train)
                
        Ypred_val = np.full((num_val,1),np.nan)
        for j in range(num_val):
            Ypred_val[j,0]=pls.predict(X_val_scaled[j,:].reshape(1,-1))          
        
        mse[i,0] = mean_squared_error(Y_val.reshape(-1), Ypred_val.reshape(-1))
    
    
    argmin_numpls = numpls[np.argmin(mse)]
    
    pls = PLSRegression(n_components = argmin_numpls)
    pls.fit(X_train_scaled, Y_train)
                
    Ypred_test = np.full((num_test,1),np.nan)
    for j in range(num_test):
        Ypred_test[j,0]=pls.predict(X_test_scaled[j,:].reshape(1,-1))          
              
    
    return Ypred_test, argmin_numpls




In [27]:

# =========================================================================
#  elastic-net, Loss : mse + penalty, 94 + dummy variable(no intersection term), hyperparameter tuning
# ========================================================================= 

def elastic_net(X,Y,num_t_v):
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np
    from sklearn.linear_model import ElasticNetCV
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import PredefinedSplit
    
    num_train = num_t_v[0]
    num_val = num_t_v[1]
    num_test = X.shape[0] - (num_train + num_val)
    
    # Split data into training and test
    X_train = X[:(num_train+num_val),:]   # train + validation
    Y_train = Y[:(num_train+num_val),:]   # train + validation
    
    X_test = X[(num_train+num_val):,:]
    
       
    # Scale Inputs for Training
    X_scaler = StandardScaler()
    X_train_scaled = X_scaler.fit_transform(X_train)

    X_test_scaled = X_scaler.transform(X_test)
    
    # pre-define validation 
    test_fold =  np.concatenate(((np.full((num_train),-1),np.full((num_val),0))))
    ps = PredefinedSplit(test_fold.tolist())
    
    # fit & predict 
    model = ElasticNetCV(cv=ps, max_iter=5000, n_jobs=-1, l1_ratio=[.1, .3, .5, .7, .9], 
                         random_state=42)
    model = model.fit(X_train_scaled, Y_train.reshape(-1))
    
    Ypred_test = np.full((num_test,1),np.nan)
    for j in range(num_test):
        Ypred_test[j,0]=model.predict(X_test_scaled[j,:].reshape(1,-1))
        
    
    return Ypred_test



# =========================================================================
#   Generalized-linear, 94 + dummy variable(no intersection term), Use cross-validation to select the number of PCA components  
# =========================================================================
# Loss ftn : MSE
# We use Lasso (Not group Lass) 
# include spline series of order 2 
# number of knots = [3,5,7...] and we choose the only one that minimize cross-validation MSE 
# we set knots by using linspace(col.mean-2*col.std, col.mean+2*col.std, # knots)
# for example if we use 3 knots, the # of variables is 94(order1) + 94*3(order 2) + dummy(74) = 450 

def general_linear(X,Y,num_t_v, num_knots):
    # num_t_v (list) : # of training set & cross-val set   ex[100, 10]
    # X consists of Traing, Val and Test set
    
    import numpy as np 
    from sklearn.metrics import mean_squared_error
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import PredefinedSplit
    from sklearn.linear_model import LassoCV
    
    num_train = num_t_v[0]
    num_val = num_t_v[1]
    num_test = X.shape[0] - (num_train + num_val)
       
    mse = np.full((len(num_knots),1),np.nan)
    Ypred_test = np.full((len(num_knots),num_test,1),np.nan)
    
    for i in range(len(num_knots)):
        
        X_temp = X
        
        # 94 variables > make spline series of order 2
        for j in range(94):
            
            # make knots
            std_train = np.std(X[:num_train,j])
            mean_train = np.mean(X[:num_train,j])           
            
            knots = np.linspace(mean_train-2*std_train, mean_train+2*std_train, num_knots[i])
            
            # add (variable - knots)**2 column
            for k in knots:
                add_col = ((X[:,j]-k)**2).reshape(-1,1)
                X_temp = np.concatenate((X_temp, add_col), axis=1)
        
        print(X_temp.shape)
        
        # Split data into training and test
        X_train = X_temp[:(num_train+num_val),:]   # train + validation
        Y_train = Y[:(num_train+num_val),:]   # train + validation
        
        X_test = X_temp[(num_train+num_val):,:]
        
        # Scale Inputs for Training
        X_scaler = StandardScaler()
        X_train_scaled = X_scaler.fit_transform(X_train)
        
        X_test_scaled = X_scaler.transform(X_test)
        
        # pre-define validation 
        test_fold =  np.concatenate(((np.full((num_train),-1),np.full((num_val),0))))
        ps = PredefinedSplit(test_fold.tolist())
        
        # we use cross-val to find best 'alpha'(penalty term in loss function)
        model = LassoCV(cv=ps, max_iter=10000, n_jobs=-1, random_state= 50)
        model = model.fit(X_train_scaled, Y_train.reshape(-1))

        
        # to choose # of knots, calculate mse of validation set
        Ypred_val = np.full((num_val,1),np.nan)
        for j in range(num_val):
            Ypred_val[j,0]=model.predict(X_train_scaled[num_train+j,:].reshape(1,-1))
            
        mse[i,0] = mean_squared_error(Y[num_train:(num_train+num_val),:].reshape(-1), Ypred_val.reshape(-1))
        
        # predic test set 
        for j in range(num_test):
            Ypred_test[i,j,0]=model.predict(X_test_scaled[j,:].reshape(1,-1))
    
     
    # choose knots that minimize mse in validation
    argmin_index = np.argmin(mse)
    
    print(argmin_index)
    
    Ypred_test_final = Ypred_test[argmin_index,:,:].reshape(-1,1)
    
    return Ypred_test_final
    
    
    

In [10]:
date_col = new_firm_data['DATE']
result = Counter(date_col)
date = list(result.keys())
num = list(result.values())

In [11]:
# X_no_inter (168) : 195703 ~ 201612, y(1) :195704 ~ 201701
# X-195703 & y-195704 are in the same row. 

X_no_inter = new_firm_data.iloc[:,2:170]     # without intersect terms
y = new_firm_data.iloc[:,-1]

In [12]:

num_est = 1  # We estimate parameter 1 time, here not use

# if we estimate parameters more than 1 time(i.e using longer data), we should set below # recursively
# Train 
# Validation 
# Test 
num_train = sum(num[84:(216+12*5)])
num_val = sum(num[(216+12*5):(216+144+60)])
num_test = sum(num[(216+144+60):])
num_t_v = [num_train, num_val]

In [13]:
y_true = y.iloc[-num_test:].to_numpy().reshape(-1,1)  # for caluclating R2oos

In [14]:

# Computational Ressources: Determine Number of available cores
ncpus = mp.cpu_count()
print("CPU count is: "+str(ncpus))



CPU count is: 4


In [22]:

from sklearn.linear_model import LinearRegression

X_train_ols = X_no_inter.iloc[:(num_train+num_val),:].to_numpy()
y_train_ols = y.iloc[:(num_train+num_val)].to_numpy()

reg = LinearRegression().fit(X_train_ols, y_train_ols)

Y_pred_ols = np.full((num_test,1),np.nan, dtype = np.float32 )

for i in range(num_test):
    Y_pred_ols[i,0] = reg.predict(X_no_inter.iloc[(num_train+num_val+i),:].to_numpy().reshape(1,-1))
    
print('R2OOS, MSE error - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols))    
    



R2OOS, MSE error - Linear regression without intersection terms :  -0.01913928985595703


In [23]:


# ===========================================================================
#     OLS, Loss: Huber Loss, 94 + dummy variable(no intersection term)
# ===========================================================================

from sklearn.linear_model import HuberRegressor

reg_huber = HuberRegressor(max_iter=500, alpha=0).fit(X_train_ols, y_train_ols)

Y_pred_ols_huber = np.full((num_test,1),np.nan, dtype = np.float32 )

for i in range(num_test):
    Y_pred_ols_huber[i,0] = reg_huber.predict(X_no_inter.iloc[(num_train+num_val+i),:].to_numpy().reshape(1,-1))

print('R2OOS, Huber Loss - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols_huber))


R2OOS, Huber Loss - Linear regression without intersection terms :  -0.0006206035614013672


In [15]:
NX = X_no_inter.iloc[109501:,:] 
NY = y.iloc[109501:]


In [16]:

# ===========================================================================
#    PCR, 94 + dummy variable(no intersection term), Use cross-validation to select the number of PCA components
# ===========================================================================

numpc =[3,4,5,10,20,25]


Y_pred_pca, argmin_numpc = FL.Pca_regression(NX.to_numpy(), NY.to_numpy().reshape(-1,1), numpc, num_t_v)

print('R2OOS, Principal Components Regression - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pca)) 
print('# of principal components : ', argmin_numpc)




R2OOS, Principal Components Regression - without intersection terms :  -0.015273094177246094
# of principal components :  5


In [17]:

# ===========================================================================
#    PLS, 94 + dummy variable(no intersection term), Use cross-validation to select the number of components
# ===========================================================================

numpls = [3,4,5,10,20,25]

Y_pred_pls, argmin_numpls = FL.Pls_regression(NX.to_numpy(), NY.to_numpy().reshape(-1,1), numpls, num_t_v)

print('R2OOS, Partial Least Square - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pls)) 
print('# of components : ', argmin_numpls)



R2OOS, Partial Least Square - without intersection terms :  -1.157390536072052
# of components :  3


In [ ]:
Y_pred_pls

array([[ 0.09239537],
       [ 0.12443741],
       [ 0.11558041],
       ...,
       [ 0.0182573 ],
       [ 0.81252609],
       [-0.00791839]])

In [18]:

# =========================================================================
#  elastic-net, Loss : mse + penalty, 94 + dummy variable(no intersection term), hyperparameter tuning
# =========================================================================

Y_pred_elastic = FL.elastic_net(NX.to_numpy(), NY.to_numpy().reshape(-1,1), num_t_v)

print('R2OOS, Elastic-net - without intersection terms : ', FL.R2OOS(y_true, Y_pred_elastic))



R2OOS, Elastic-net - without intersection terms :  0.0024675253993113877


In [ ]:


# =========================================================================
#   Generalized-linear, 94 + dummy variable(no intersection term), Use cross-validation to select the number of PCA components  
# =========================================================================
# Loss ftn : MSE
# We use Lasso (Not group Lass) 
# include spline series of order 2 
# number of knots = [3,5,7...] and we choose the only one that minimize cross-validation MSE 
# we set knots by using linspace(col.mean-2*col.std, col.mean+2*col.std, # knots)
# for example if we use 3 knots, the # of variables is 94(order1) + 94*3(order 2) + dummy(74) = 450 

num_knots = [3]

Y_pred_general_lin = general_linear(NX.to_numpy(), NY.to_numpy().reshape(-1,1), num_t_v, num_knots)

print('R2OOS, generalized linear - without intersection terms / with knots : ', FL.R2OOS(y_true, Y_pred_general_lin))




(3600408, 450)


In [24]:
print('R2OOS, Huber Loss - Linear regression without intersection terms : ', FL.R2OOS(y_true, Y_pred_ols_huber))
print('R2OOS, Principal Components Regression - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pca)) 
print('R2OOS, Partial Least Square - without intersection terms : ', FL.R2OOS(y_true, Y_pred_pls)) 
print('R2OOS, Elastic-net - without intersection terms : ', FL.R2OOS(y_true, Y_pred_elastic))
print('R2OOS, generalized linear - without intersection terms / with knots : ', FL.R2OOS(y_true, Y_pred_general_lin))

R2OOS, Huber Loss - Linear regression without intersection terms :  -0.0006206035614013672
R2OOS, Principal Components Regression - without intersection terms :  -0.015273094177246094
R2OOS, Partial Least Square - without intersection terms :  -1.157390536072052
R2OOS, Elastic-net - without intersection terms :  0.0024675253993113877
R2OOS, generalized linear - without intersection terms / with knots :  0.001908131940232649


In [26]:
Y_pred_general_lin

array([[0.00608327],
       [0.00608327],
       [0.00608327],
       ...,
       [0.00608327],
       [0.00608327],
       [0.00608327]])

In [25]:
Y_pred_elastic

array([[0.00522649],
       [0.00624618],
       [0.00636093],
       ...,
       [0.00365887],
       [0.0046813 ],
       [0.0039413 ]])